# Correlation used to determine formation temperature from system pressure

#### ![](newcorrelation.PNG)

#### Reference: Mohammad ali Ghayyem, Mojtaba Izadmehr*, Rauf Tavakoli,2014,Developing a simple and accurate correlation for initial estimation of hydrate formation temperature of sweet natural gases using an eclectic approach,Journal of Natural Gas Science and Engineering, 21, 1-4

##### t is in °F and P is in psia

# Bukacek correlation used to determine water content in sweet gas

###### ![](bukacek.PNG)

###### ![](saturatedvapourpressure.PNG)

#### P is in between 1 and 960 bar and T is in between 288.15 K and 511.15 K

#### Reference: Alexandre TERRIGEOL , Christophe COQUELET, Antonin CHAPOY,2015,Water content assessment in acid gas,GPA Europe, Annual Conference, 7-8

##### P is in MPa and T is in °C

# Maddox correlation used to determine water content in sour gas

###### ![](maddox.PNG)

##### P is in psia and T is in °F

###### ![](mt.PNG)

#### Reference: Natural Gas Hydrates, 2nd Edition 2009 - A Guide for Engineers (Malestrom),237-241

# Hammerschimdt correlation used to determine weight fraction of inhibitor

###### ![](hsqw.PNG)

##### T is in K, Ks = 1861

#### Reference: Natural Gas Hydrates, 2nd Edition 2009 - A Guide for Engineers (Malestrom),113-115

## Importing the Datasets and Libraries and defining functions

In [2]:
import pandas as pd
import math
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
dfcd = pd.read_excel('maddoxcd.xlsx')
dfhs = pd.read_excel('maddoxhs.xlsx')

In [3]:
unit = widgets.Dropdown(
       options=['MPa', 'psia', 'atm'],
       description='Unit: ')

pressurein = widgets.Text(
       value='',
       description='Pressure: ', )

tempin = widgets.Text(
       value='',
       description='Source T: ', )

tempout = widgets.Text(
       value='',
       description='Site T: ', )
tunit = widgets.Dropdown(
       options=['F', 'C', 'K'],
       description='Unit: ')

inhitwf = widgets.Text(
       value='',
       description='Weight fraction: ', )
inhit = widgets.Dropdown(
       options=['Ethylene Glycol', 'Methanol'],
       description='Inhibitor: ')

In [4]:
def systemprature(gasgravity,pressure,unit):
    if(unit=='MPa'):
        ppsia = pressure*145.038*1.0
        pmpa = pressure
        pressure = pressure*145.038*1.0
    elif(unit=='atm'):
        ppsia = pressure*14.7
        pmpa = pressure/10.0
        pressure = pressure*14.7
    elif(unit=='psia'):
        ppsia = pressure
        pmpa = pressure/(145.038)
        pressure = pressure
    Temprature = (-26.115) + ((-23.728)/(gasgravity*1.0)) + ((23.942)*(math.log(pressure*1.0))) + ((-0.738)*(math.exp((gasgravity)**(-2.3)))) + ((-1.135)*((math.log(pressure*1.0))**2)) + ((0.443)*(math.log(pressure*1.0))*(math.exp((gasgravity)**(-1.7))))
    tink = (Temprature-32)*(5.0/9.0) + 273.15
    return Temprature,pmpa,tink,ppsia
def buckekinsweetgas(t,p):
    tk = t+273.15 #tk in K
    #psw = 0.61094*math.exp((17.625*t)/(t+243.04))*(0.001)
    psw1 = (10**(-6))*math.exp(73.649- ((7258.2)/tk)-(7.3037*(math.log(tk)))+ (4.1653*(10**(-6))*(tk**2)))
    tf = (t*1.8) + 32#tf in F
    b = (-3083.87/(491.6 + tf))  + 6.69449
    wc = (47484*(psw1/p)) + math.exp(b)
    return wc
def interpolation(df,t):
    if(t<80):
        print("Error")
    else:        
        for i in range(len(df['T'])):
            if(t==df['T'][i]):
                a0 = df['a0'][i]
                a1 = df['a1'][i]
                a2 = df['a2'][i]
                break
            elif(t<df['T'][i]):
                x1 = df['T'][i-1]
                x2 = df['T'][i]
                a0y1 = df['a0'][i-1]
                a0y2 = df['a0'][i]
                a1y1 = df['a1'][i-1]
                a1y2 = df['a1'][i]
                a2y1 = df['a2'][i-1]
                a2y2 = df['a2'][i]

                a0 = ((a0y2-a0y1)/((x2-x1)*1.0))*(t-x1) + a0y1
                a1 = ((a1y2-a1y1)/((x2-x1)*1.0))*(t-x1) + a1y1
                a2 = ((a2y2-a2y1)/((x2-x1)*1.0))*(t-x1) + a2y1
                break
    return a0,a1,a2
#t in F, p in psia
def maddox(t,p,co2,h2s,hc,sweetwc):
    a0h2s,a1h2s,a2h2s = interpolation(dfhs,t)
    a0co2,a1co2,a2co2 = interpolation(dfcd,t)
    whs = 10**(a0h2s  + (a1h2s)*math.log(p,10) + a2h2s*(math.log(p,10)**2))
    wcd = 10**(a0co2  + (a1co2)*math.log(p,10) + a2co2*(math.log(p,10)**2))
    wc = (hc*sweetwc) + (wcd*co2) + (whs*h2s)
    return wc
def wtfracofinhibitor(tout,t,molin):
    delt = (t - tout) + 5.0
    w  =  ((delt*molin)/(1861.0+(delt*molin*1.0)))
    return w

### Enter the Mole fractions of Components

In [5]:
sum = 0
while(sum!=1):
    Methane = float(input("Methane: ")or"0")
    Ethane = float(input("Ethane: ")or"0")
    Propane = float(input("Propane: ")or"0")
    Isobutane = float(input("Isobutane: ")or"0")
    Ethylene = float(input("Ethylene: ")or"0")
    Propylene = float(input("Propylene: ")or"0")
    Nitrogen = float(input("Nitrogen: ")or"0")
    H2S = float(input("Hydrogen Sulphide: ")or"0")
    CO2 = float(input("Carbon Dioxide: ")or"0")
    sum = Methane + Ethane + Propane + Isobutane + Ethylene + Propylene + Nitrogen + H2S + CO2
    if(sum!=1):
        print("\nInvalid input ! Enter again.")

Methane: 
Ethane: 
Propane: 
Isobutane: 0.6
Ethylene: 
Propylene: 
Nitrogen: 
Hydrogen Sulphide: 0.4
Carbon Dioxide: 


### Input System Pressure of the Pipeline

In [6]:
display = widgets.HBox([pressurein, unit])
display

### Input flow rate of Gas

In [7]:
Q = float(input("Flow rate gas (Qg) (MMSCFD): "))

Flow rate gas (Qg) (MMSCFD): 100


### Input Source and Site Temprature

In [8]:
dis = widgets.VBox([tempin, tempout, tunit ])
dis

### Input Inhibitor and its Weight fraction

In [108]:
d = widgets.HBox([inhit, inhitwf ])
d

### Gas specific gravity and hydrocarbon mole fraction

In [9]:
hcmolfrac = Methane + Ethane + Propane + Isobutane + Ethylene + Propylene + Nitrogen
gasmolweight = (16.0*Methane) + (30.0*Ethane) + (44.0*Propane) + (58.0*Isobutane) + (28.0*Ethylene)+ (42.0*Propylene)+ (44.0*CO2)+ (34.0*H2S) + (28*Nitrogen)  
gasgravity = (gasmolweight*1.0)/(28.9)
print("Gas specific gravity: " + str(gasgravity))

Gas specific gravity: 1.6747404844290659


### Gas Hydrate formation temprature

In [10]:
p = float(pressurein.value)
u = unit.value
t,pmpa,tink,ppsia = systemprature(gasgravity,p,u)
print("Gas Hydrate formation temperature: "+str(t) + "°F")

Gas Hydrate formation temperature: 78.01693069206526°F


##### Converting input tempratures to °C

In [11]:
tin = float(tempin.value)
tout = float(tempout.value)
Tunit = tunit.value

In [12]:
if(Tunit=='C'):
    tinc = tin
    toutc = tout
    toutk = toutc + 273.15
    tinf = (tinc*1.8) + 32.0
    toutf = (toutc*1.8) + 32.0
elif(Tunit=='F'):
    tinf = tin
    toutf = tout
    tinc = (tinf-32)*(5.0/9.0)
    toutc = (toutf-32)*(5.0/9.0)
    toutk = toutc + 273.15
elif(Tunit=='K'):
    tinc = tin - 273.15
    toutc = tout -273.15
    toutk = toutc + 273.15
    tinf = (tinc*1.8) + 32.0
    toutf = (toutc*1.8) + 32.0


##### Check if hydrate formation inside pipe

In [113]:
if(toutf<t<tinf):
    print('Gas hydrate will form inside pipe.')
else:
    print("Gas hydrate won't form inside pipe. Change inlet and outlet temperature.")

Gas hydrate will form inside pipe.


### Total water content inside pipe

In [14]:
sweetin = buckekinsweetgas(tinc,pmpa)
sweetout = buckekinsweetgas(toutc,pmpa)
if(hcmolfrac==1):
    twc = (sweetin - sweetout)*(Q/24.0)
    inwc = sweetin
    outwc = sweetout
else:
    maddoxin = maddox(tinf,ppsia,CO2,H2S,hcmolfrac,sweetin)
    maddoxout = maddox(toutf,ppsia,CO2,H2S,hcmolfrac,sweetout)
    twc = (maddoxin - maddoxout)*(Q/24.0)
    inwc = maddoxin
    outwc = maddoxout
print("Inlet water content: " + str(inwc))
print("Outlet water content: " + str(outwc))

Inlet water content: 76.34570630069572
Outlet water content: 76.34570630069572


### Weight fraction of Inhibitor that is needed to be added

In [117]:
if(inhit.value =='Ethylene Glycol' ):
    mm = 62.07
else:
    mm = 32.04
w = wtfracofinhibitor(toutk,tink,mm)
print("Weight fraction of inhibitor required: "+str(w))

Weight fraction of inhibitor required: 0.3713244400687442


### Mass flow rate of Inhibitor that is needed to be added

In [118]:
wf = float(inhitwf.value)
Y = ((w*twc)/(wf-w))
print("Total flow rate of inhibitor required: "+str(round(Y,2)))

Total flow rate of inhibitor required: 3.94


## Summarising all the output

In [120]:
variables = ['System Pressure', 'Formation Temperature', 'Inlet Temperature','Outlet Temperature','Inlet Water Content','Outlet Water Content', 'Total Water Content','Gas Flow Rate', 'Inhibitor','Inhibitor Weight fraction required','Mass flow rate of Inhibitor']
df = pd.DataFrame(index=variables,columns = ['Value'])
df['Value']['System Pressure'] = " "+str(round(p,2)) + " psia "
df['Value']['Formation Temperature'] =" "+ str(round(t,2)) + " °F "
df['Value']['Inlet Temperature'] =" "+ str(round(tin,2)) + " °" + Tunit + " "
df['Value']['Outlet Temperature'] =" "+ str(round(tout,2)) + " °"+Tunit + " "
df['Value']['Inlet Water Content'] =" "+ str(round(inwc,2)) + " lb/MMSCF "
df['Value']['Outlet Water Content'] =" "+ str(round(outwc,2)) + " lb/MMSCF "
df['Value']['Total Water Content'] =" "+ str(round(twc,2)) + " lb/hr "
df['Value']['Gas Flow Rate'] =" "+ str(round(Q,2)) + " MMSCFD "
df['Value']['Inhibitor'] =" "+ inhit.value
df['Value']['Inhibitor Weight fraction required'] =" "+ str(round(w,2))
df['Value']['Mass flow rate of Inhibitor'] = " "+str(round(Y,2)) + " lb/hr "
df

,Value
System Pressure,2000.0 psia
Formation Temperature,62.88 °F
Inlet Temperature,85.0 °F
Outlet Temperature,40.0 °F
Inlet Water Content,18.01 lb/MMSCF
Outlet Water Content,5.33 lb/MMSCF
Total Water Content,4.23 lb/hr
Gas Flow Rate,8.0 MMSCFD
Inhibitor,Ethylene Glycol
Inhibitor Weight fraction required,0.37
